#Trabalho 1
##Filas e Ruína do Apostador
##Modelagem e Avaliação de Desempenho

## Entrega Preliminar

### Alunos:
Daniel Corcino de Albuquerque - DRE: 118188457

Letícia Freire Carvalho de Sousa - DRE: 118025324

Lucas Favilla Ferreira Alves da Silva - DRE: 119156518

Roberto Leonie Ferreira Moreira - DRE: 116062192



#Ruína do Apostador

Abaixo executamos algumas simulações computando se primeiro a fila esvazia (equivalente a ruína do apostador), se a fila enche, no caso das filas finitas (fortuna do apostador) ou se a fila nunca esvazia durante o tempo de simulação (no caso das infinitas).

No caso das finitas só paramos a simulação quando a fila enche ou esvazia.

Colhemos as métricas relacionadas ao número médio de passos até a fila esvaziar e também ao tempo esperado para tanto.



In [ ]:
import numpy as np
import random
import copy
import time
from queue import PriorityQueue

# A simulacao em si
def epidemic_simulation(LAMBDA, MU, LIM_TIME, DETERMINISTIC = 0, VERBOSE = False):
  # classe que modela os eventos de chegada e saída de cada cliente no sistema
  # id -- id do cliente -- incremental a partir de 1
  # event_type -- tipo do evento ("A" - chegada, "D" - saída)
  # arrival_time -- momento da chegada no sistema
  # departure_time -- momento de saída do sistema
  # service_start -- momento em que começa a ser serviço
  # time -- momento atual, pode ser igual a arrival_time ou departure_time, a depender
  #         de qual evento está sendo modelado


  class Event:
    def __init__(self, event_type, time, id):
      self.event_type = event_type
      self.time = time
      self.service_start = -1
      if(event_type == 'A'):
        self.arrival_time = self.time
        self.departure_time = -1
      else:
        self.departure_time = self.time
      self.id = id

    # comparador, utilizado para sabermos que evento vem primeiro
    # em caso de empate damos prioridade para a saída do sistema
    def __lt__(self, other):
      if(self.time<other.time):
          return True
      if(self.time>other.time):
          return False
      else:
          return (self.event_type == 'D')

  # o tempo de vida residual é modelado por uma variavel aleatória exponencial com taxa MU
  def residual_life():
    if(DETERMINISTIC != 0):
      return DETERMINISTIC
    return random.expovariate(MU)

  # o momento da proxima chegada é modelado pela soma de variaveis aleatórias exponenciais iid com taxa LAMBDA
  ARRIVAL_TIME = 0
  def next_arrival():
    return ARRIVAL_TIME + random.expovariate(LAMBDA)

  # classe para modelar o sistema
  # waiting_queue -- lista que representa a fila de espera (células são do tipo Event)
  # server -- variável do tipo Event que representa o cliente que está sendo servido
  class System:
    def __init__(self):
      self.waiting_queue = []
      self.server = None

    # cliente e entra na lista de espera
    def arrival(self, e):
      self.waiting_queue.append(e)

    # cliente que está sendo servido sai do sistema
    # garantidamente só é chamado se o sistema estava ocupado
    def departure(self):
      server = copy.deepcopy(self.server)
      self.server = None
      return server

    # o próximo cliente sai da fila de espera e é servido
    def next(self, T):
      if(self.busy() or not(self.waiting_queue)):
        return None
      self.server = self.waiting_queue.pop(0)
      x = residual_life()
      self.server.departure_time = T + x
      self.server.time = self.server.departure_time
      return self.server

    # verifica se o sistema está ocupado
    def busy(self):
      return not(self.server == None)

    # tamanho da fila de espera
    def waiting_queue_size(self):
      return len(self.waiting_queue)

    # tamanho da fila (fila de espera + servidor)
    def full_queue_size(self):
      s = len(self.waiting_queue)
      if(self.busy()):
        s += 1
      return s

  class Tree:
    def __init__(self):
      self.parent = []
      self.height = []
      self.max_out_degree = 0
      self.root = None
      self.children = []
      self.out_degree_frequence = {}

    def dfs(self, v):
      if len(self.children[v]) not in self.out_degree_frequence.keys():
        self.out_degree_frequence[len(self.children[v])] = 0
      self.out_degree_frequence[len(self.children[v])] += 1
      self.max_out_degree = max(self.max_out_degree, len(self.children[v]))
      for c in self.children[v]:
        self.dfs(c)
        self.height[v] = max(self.height[v], 1 + self.height[c])

    def calc(self):
      self.children = [[] for i in range(len(self.parent))]
      self.height = [0 for i in range(len(self.parent))]

      for u in range(len(self.parent)):
        if(self.parent[u] == -1):
          self.root = u
        else:
          self.children[self.parent[u]].append(u)

      self.dfs(self.root)

    def E_height_nodes(self):
      h = 0
      for v in range(len(self.height)):
        h += self.height[v]
      return h / len(self.height)




  gambler_ruin = -1

  # variáveis principais
  system = System()
  eventQueue = PriorityQueue()
  tree = Tree()

  id = 0 # id do próximo cliente que chegará

  ARRIVAL_TIME = 0
  # evento de chegada do próximo cliente
  # é inicializado fora do laço principal para que a simulção comece com um cliente no sistema
  eventQueue.put(Event('A', ARRIVAL_TIME, id))

  T = 0 # tempo atual
  P = 0 # número de passos

  # laço principal da simulação
  # Quando a simulação chega ao tempo limite paramos a simulação
  while(T < LIM_TIME and gambler_ruin == -1):

    # evento atual, pode ser de chegada ou de saída
    current_event = eventQueue.get()

    # atualizamos tempo atual da simulação
    last_T = T
    T = current_event.time

    # atualizamos número de passos
    P += 1

    # se o evento atual é de chegada
    if(current_event.event_type == "A"):

      if(system.busy()):
        tree.parent.append(system.server.id)
      else:
        tree.parent.append(-1)

      id += 1
      ARRIVAL_TIME = next_arrival()
      next_arrival_event = Event('A', ARRIVAL_TIME, id)
      eventQueue.put(next_arrival_event)
      system.arrival(current_event)


    # se o evento atual é de partida
    else:
      # cliente termina de ser servido
      system.departure()

    serving = system.next(T)
    # se um cliente começou a ser servido neste momento
    if(serving):

      # gerando o evento de saída do cliente do sistema
      serving.service_start = T
      serving.event_type = 'D'
      next_departure_event = serving
      eventQueue.put(next_departure_event)

    # se o sistema está ocioso
    if(not(system.busy())):
      gambler_ruin = 1 # apostador cai em ruina
      break

  tree.calc()


  if(VERBOSE):
    print("Métricas das árvores")
    #print("Pais de cada vértice:")
    #print(tree.parent)
    print("\nDistribuição dos graus de saída: ")
    print(dict(sorted(tree.out_degree_frequence.items())))
    print(f"\nGrau saída raíz =  {len(tree.children[tree.root])}")
    print(f"Grau saída máximo =  {tree.max_out_degree:.3f}")
    print(f"Altura árvore =  {tree.height[tree.root]:.3f}")
    print(f"Média altura vértices =  {tree.E_height_nodes():.3f}")

  return [tree, gambler_ruin, T, P]


In [ ]:
epidemic_simulation(2, 1, 100000, 0, True)

Métricas das árvores

Distribuição dos graus de saída: 
{0: 1}

Grau saída raíz =  0
Grau saída máximo =  0.000
Altura árvore =  0.000
Média altura vértices =  0.000


[<__main__.epidemic_simulation.<locals>.Tree at 0x7feec6bfea70>,
 1,
 0.3548113414165023,
 2]

In [ ]:
epidemic_simulation(2, -1, 100000, 0.1, True)

Métricas das árvores

Distribuição dos graus de saída: 
{0: 1}

Grau saída raíz =  0
Grau saída máximo =  0.000
Altura árvore =  0.000
Média altura vértices =  0.000


[<__main__.epidemic_simulation.<locals>.Tree at 0x7feec6bfc1c0>, 1, 0.1, 2]

In [ ]:
epidemic_simulation(2, -1, 100000, 0.2, True)

Métricas das árvores

Distribuição dos graus de saída: 
{0: 1}

Grau saída raíz =  0
Grau saída máximo =  0.000
Altura árvore =  0.000
Média altura vértices =  0.000


[<__main__.epidemic_simulation.<locals>.Tree at 0x7feec6bfca30>, 1, 0.2, 2]

In [ ]:
epidemic_simulation(2, -1, 100000, 0.5, True)

Métricas das árvores

Distribuição dos graus de saída: 
{0: 2, 2: 1}

Grau saída raíz =  2
Grau saída máximo =  2.000
Altura árvore =  1.000
Média altura vértices =  0.333


[<__main__.epidemic_simulation.<locals>.Tree at 0x7feec6bfc5b0>, 1, 1.5, 6]

In [ ]:
epidemic_simulation(2, -1, 100000, 1, True)

Métricas das árvores

Distribuição dos graus de saída: 
{0: 114116, 1: 26965, 2: 27037, 3: 18083, 4: 8973, 5: 3696, 6: 1252, 7: 372, 8: 79, 9: 17, 10: 2, 11: 1}

Grau saída raíz =  1
Grau saída máximo =  11.000
Altura árvore =  19.000
Média altura vértices =  0.836


[<__main__.epidemic_simulation.<locals>.Tree at 0x7feec6bfce80>,
 -1,
 100000,
 300593]

In [ ]:
def epidemic_experiment(LAMBDA, MU, COUNT_SIMULATIONS = 20, LIM_TIME = 1000000):
  averages = {"root_out_degree": 0.0, "max_out_degree": 0.0, "tree_height":0.0, "nodes_height":0.0, "time": 0.0, "count_nodes": 0.0}
  for sim in range(COUNT_SIMULATIONS):
    #print(f"Simulação {sim}:\n")
    tree, gambler_ruin, T, P = epidemic_simulation(LAMBDA, MU, LIM_TIME)
    averages["root_out_degree"] += len(tree.children[tree.root])
    averages["max_out_degree"] += tree.max_out_degree
    averages["tree_height"] += tree.height[tree.root]
    averages["nodes_height"] += tree.E_height_nodes()
    averages["time"] += T
    averages["count_nodes"] += len(tree.parent)

  averages["clients"] = averages["count_nodes"] / averages["time"]

  for key, value in averages.items():
    if(key == "clients"):
      continue
    value /= COUNT_SIMULATIONS

  print("\nResultado:\n")

  print(averages)


In [ ]:
epidemic_experiment(1, 2)

In [ ]:
epidemic_experiment(2, 4)

In [ ]:
epidemic_experiment(1.05, 1)

In [ ]:
epidemic_experiment(1.10, 1)

Simulação 0:

Simulação 1:

Simulação 2:

Simulação 3:

Simulação 4:

Simulação 5:

Simulação 6:

Simulação 7:

Simulação 8:

Simulação 9:

Simulação 10:

Simulação 11:

Simulação 12:

Simulação 13:

Simulação 14:

Simulação 15:



KeyboardInterrupt: ignored